A.S. Lundervold, v041022

# Introduction

<img width=60% src="https://github.com/alu042/DAT158-2022/raw/main/notebooks/assets/random_forest_detailed.png">

Random forests are **ensembles** of decision trees. As we've seen, a single decision tree is typically susceptible to small changes in the training data, and they tend to overfit. However, we can mitigate these problems by constructing multiple decision trees in a particular way and then combining them to make predictions. As well as being less sensitive and less prone to overfitting, the resulting models, called **random forests**, have much higher performance. 

Ensembling techniques are related to the concept of the <a href="https://en.wikipedia.org/wiki/Wisdom_of_the_crowd">wisdom of the crowd</a>. In many situations, a crowd of non-expert that combines their predictions outperform individual experts.

The idea is not new, of course. Here's Aristotle pointing this out back in 350 B.C.E:
> *For the many, of whom each individual is but an ordinary person, when they meet together may very likely be better than the few good, if regarded not individually but collectively, just as a feast to which many contribute is better than a dinner provided out of a single purse* <br>&ndash; Aristotle, <a href="http://classics.mit.edu/Aristotle/politics.3.three.html">Politics Book III</a>

*Ensembling* is a general technique for combining machine learning models to make even more powerful models. 

There are several variants of this: for example **voting**, **bagging**, **boosting**, **stacking**. 

A very powerful and widely used model made using ensembling are **random forests**, based on what's called **bagging**.

# Setup

In [ ]:
# This is a quick check of whether the notebook is currently running on Google Colaboratory
# or on Kaggle, as that makes some difference for the code below.
# We'll do this in every notebook of the course.
try:
    import colab
    colab=True
except:
    colab=False

import os
kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [ ]:
# To display plots directly in the notebook:
%matplotlib inline

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import matplotlib
import sklearn
from pathlib import Path
import seaborn as sns

In [ ]:
# To make the notebook reproducible
seed = 42
np.random.seed(seed)

In [ ]:
# Set the directory in which to store data
NB_DIR = Path.cwd()       # Set NB_DIR to be the current working directory
DATA = NB_DIR/'data'      # The data dir is the subdirectory 'data' under NB_DIR

DATA.mkdir(exist_ok=True) # Create the data directory

# Random forests

As we've seen, single decision trees tend to be very sensitive to the exact training data it's fed and also tend to overfit.
By training multiple decision trees on different parts of the training data, each one may overfit and be very sensitive. Still, by averaging their predictions, the sensitivity is reduced and the overfitting combatted.

Let's say we want to build 500 trees on a training set containing n_samples points. Each tree is built in the following way:

1. Each tree is trained on different data sets: Choose one data point at random for the training data, **with replacement**. Do this `n_samples` times. That is, until you have `n_samples` points. This is called **bootstrap sampling** or **bagging**.
2. Each tree trained on different features: Train a decision tree on this data set, but at each node of the tree, **select a random set of features to consider when splitting**, up to `max_features` of them.  

This is a random forest.

Predictions from a random forest are made differently for regression and classification: 

- **Regression:** Let each tree make a prediction, then average them.
- **Classification:** Let each tree make a prediction and use a **soft voting strategy** to combine them. As we saw in the decision trees notebook, each tree provides a probability for its prediction. Average these probabilities and predict the class that has the highest average probability. 

<img width=60% src="https://github.com/alu042/DAT158-2022/raw/main/notebooks/assets/random_forest_detailed.png">

## Two simple examples of using random forests

Let's try it out on the two data sets studied in the notebook on decision trees: the **diabetes data set** (classification) and the **Boston housing data** set (regression).

### Classification: the diabetes data set

In [ ]:
diabetes_data = pd.read_csv('https://assets.datacamp.com/production/course_1939/datasets/diabetes.csv')
diabetes_data.head()

In [ ]:
X = diabetes_data.drop('diabetes', axis=1)
y = diabetes_data['diabetes']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=seed)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=seed)

In [ ]:
rf_clf.fit(X_train, y_train)

Random forests have some hyperparameters, mostly inherited from those for decision trees:

In [ ]:
rf_clf.get_params()

The most important ones when using random forests in practice are:
* the parameters acting as regularization for the decision trees discussed in the previous notebook: i.e. `max_depth` (arguably the most important one, and often the only regularization set to non-default values), `max_features`, `min_samples_split`, `min_samples_leaf`, `max_leaf_nodes`, `min_impurity_decrease`, `min_weight_fraction_leaf`.
* `n_estimators`: the number of decision trees in the forest. Increase to get increased model expressiveness. 
* `n_jobs`: set to `-1` to use all available CPUs. Useful when training random forests on large data sets.

Let's test the trained model on the test set:

In [ ]:
rf_clf.score(X_test, y_test)

### Regression: the California housing data set

In [ ]:
from sklearn.datasets import fetch_california_housing

In [ ]:
housing = fetch_california_housing(data_home='./data')

In [ ]:
X = pd.DataFrame(data=housing.data, columns=housing.feature_names)
y = housing.target

In [ ]:
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(random_state=seed)

In [ ]:
rf_reg.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
y_pred = rf_reg.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred)

In [ ]:
mean_absolute_error(y_test, y_pred)

# Voting and model averaging 

The voting ideas used by the decision trees of a random forest are applicable more generally.

It's not necessary that the models in the ensemble are the same or, if they are, that they're trained on random samples of the training data.

One could, e.g., train multiple different models, for example, several polynomial regression models and multiple random forests, and then have them vote on the class or value of instances during training and inference.

Scikit-learn's `VotingClassifier` and `VotingRegressor` are helpful for this. We'll take a look at how it can be used at the end of the notebook.

Another option simpler that can give increased performance is to train a bunch of slightly different versions of the same model.

For example: here's a simple way to combine multiple random forest regressors trained on the above California housing data set:


In [ ]:
n_models = 20

# Make an array to store all the models' predictions
all_predictions = np.zeros(X_test.shape[0])

for i in range(n_models):
    rf = RandomForestRegressor(random_state=seed+i, n_jobs=-1) # Note the random state updates
    rf.fit(X_train, y_train)
    all_predictions+=rf.predict(X_test)

In [ ]:
# The final predictions are the average of the n_models models predictions:
y_pred = all_predictions/ n_models

In [ ]:
mean_squared_error(y_test, y_pred)

A slight improvement. :-)

# Boosting

> Producing strong learners from an ensemble of weak learners

## Introduction

Another widely used and powerful class of tree-based machine learning models are those based on **boosted trees**. With this tool added to your toolbelt, you're well-equipped to face most machine learning modeling tasks you'll meet in real life.

We've learned that random forests are ensembles of decision trees, making predictions by simply averaging all the models in the ensemble. **Boosting** takes another approach to ensembling: new members are added to the ensemble _sequentially_, and each new model is trained on the _errors_ of the ensemble constructed so far, iteratively learning from its mistakes. Again, we'll focus on boosting based on decision trees, as these are the most common base models.

One of the main boosting techniques is **AdaBoost**. Here each additional tree focuses on the examples (i.e., _instances_) that were misclassified by the previous trees. 

In **gradient boosting**, each tree added to the ensemble tries to predict the *residual error* from the previously added tree. In other words, it's trained to predict the difference between the correct value and the value predicted from the ensemble so far. 

Each individual tree is what's called a **weak learner**, unable to fit the data very well. But by combining them by having each tree try to predict the residual of the previous trees' predictions, the result is a very **strong learner**. In fact, we can end up with some of the strongest learners we know. 

More details about gradient boosting can be found [here](http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting/) or [here]() (we will base our discussion partly on the one in the first link).

## AdaBoost: Adaptive boosting

AdaBoost was introduced in 1995 ([here](https://www.sciencedirect.com/science/article/pii/S002200009791504X)) and was the first boosting algorithm with practical use cases. It won the authors the 2003 Gödel Prize for outstanding journal article in theoretical computer science because of the algorithm's _"elegance, simplicity of implementation, its wide applicability, and its striking success in reducing errors in benchmark applications even while its theoretical assumptions are not known to hold"_. 

AdaBoost _"set off an explosion of research in the fields of statistics, artificial intelligence, experimental machine learning, and data mining"_ ([source](http://eatcs.org/index.php/component/content/article/505)).

The basic idea is relatively simple: 

- Give each instance in the training data an _instance weight_, initially set to 1/m where m is the number of instances.
- Extract a sample of the training data and train a *weak learner* on it. The weak learners in AdaBoost are typically decision trees of depth 1 (so-called *decision stumps*), but other models can be used. 
- Calculate a weight for the learner based on its error rate. Higher accuracy means higher weight.
- Then increase the weight of the misclassified instances according to the learner's weight. An instance misclassified by a high-weight learner gets higher weight than one misclassified by a low-weight learner.
- Extract another sample from the training data, sampled according to the instance weights. 
- Train the next weak learner on the sampled data, compute the learner weight and update the weights of the training instances accordingly. 
- Continue until perfect fit or `n_estimators` have been trained. 

Each new learner added to the ensemble corrects the shortcomings of previous models as it focuses on high-weight instances. In other words, the weighing of the training data makes the new learner focus more on difficult cases.

Predictions from AdaBoost are made by producing predictions from all the weak learners, weighted according to their weights.

Here's an illustration from the textbook by A. Geron:

<img width=50% src="https://github.com/alu042/DAT158-2022/raw/main/notebooks/assets/adaboost.png">

### AdaBoost in more detail

To make the above procedure a bit clearer, here are the steps for classification using AdaBoost in more detail:

At the start, we don't know which instances are most difficult, so we give them the same weight, making sure that the sum of all weights is 1: set the weights of all $m$ instances to $w^{(i)} = 1/m$ (note that the sum of all instance weights is 1).

1. Train a weak learner on a sample of the training data
2. Get the error rate of the learner on the sample:

$$r_i = \frac{\mbox{sum of weights of errors}}{\mbox{sum of errors}} = \frac{\underset{\hat{y}_j^{(i)} \neq y_j}{\sum w^{(i)}}}{\sum w^{(i)}}$$

3. Compute the learner's weight:

$$\alpha_j = \eta \log \frac{1-r_j}{r_j},$$ where $\eta$ is the learning rate. (See below for an explanation of the formula.)

4. Update all the weights of the misclassified instances:

$$\mbox{For  } \hat{y}_j^{(i)} \neq y^{(i)}: \quad w^{(i)} := w^{(i)} \exp(\alpha_j),$$

and normalize them to keep the sum equal to 1: $$w^{(i)} := \frac{w^{(i)}}{\sum w^{(i)}}.$$

Repeat 1-4.

##### **A closer look at the learner weights:**

In [ ]:
# The error rate of a learner is between 0 and 1
r = np.arange(0.001, 1, 0.01) 

In [ ]:
# The formula for the learner weights
learning_rate = 1

plt.figure(figsize=(8,6))
plt.plot(r, learning_rate*np.log((1-r)/r))
plt.ylim([-5,5])
plt.show()

> Higher error rate means lower learner weight.

### In scikit-learn

Let's try it out on the California Housing data provided by scikit-learn:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

Let's first try a single decision stump (our weak learner):

In [ ]:
dt_reg = DecisionTreeRegressor(max_depth=1, random_state=seed)
dt_reg.fit(X_train, y_train)
mean_squared_error(y_test, dt_reg.predict(X_test))

..then an AdaBoost ensemble of 10 decision stumps:

In [ ]:
ada_reg = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=1), 
                             n_estimators=10, random_state=seed)

In [ ]:
ada_reg.fit(X_train, y_train)
mean_squared_error(y_test, ada_reg.predict(X_test))

Significantly better!

**Note:** this was an unusually simple AdaBoostRegressor, just for demonstration and comparison with decision trees. Here's a more reasonable model:

In [ ]:
ada_reg = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=None), 
                             n_estimators=50, random_state=seed)

ada_reg.fit(X_train, y_train)
mean_squared_error(y_test, ada_reg.predict(X_test))

## Gradient boosting: gradient descent + boosting

In AdaBoost, the learners added to the ensemble try to correct their predecessors by changing the weights on the data points. Gradient boosting has the same objective, but rather than tweaking the instance weights; each new learner attempts to predict the _residual error_ of the ensemble.

### Residual errors: the basic idea behind gradient boosting

We'll try to build such a boosting ensemble ourselves. First, we'll see the idea in a spreadsheet, then in code. Gradient boosting builds on this, as we shall see.

#### Residual errors and boosting using a spreadsheet

> Go to http://bit.ly/boosting_example for boosting explained in a spreadsheet.

#### Residual errors and boosting using scikit-learn

Let's use some randomly generated data.

In [ ]:
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

In [ ]:
plt.scatter(X,y)
plt.show()

We also need some new data on which to make predictions:

In [ ]:
X_new = np.linspace(-0.5, 0.5, 500)
X_new = X_new.reshape(-1, 1)

Our initial model will be the mean value of the target (as that is the value minimizing mean squared error):

In [ ]:
from sklearn.dummy import DummyRegressor

In [ ]:
F_0 = DummyRegressor(strategy='mean')

In [ ]:
F_0.fit(X, y)

This is our initial model and our goal is to improve it.

In [ ]:
fig, ax = plt.subplots()
ax.plot(X[:, 0], y, 'b.')
ax.plot(X_new, F_0.predict(X_new), 'g-')
plt.show()

Next we'll train a single decision tree on the residuals of the initial model:

In [ ]:
y_0 = y - F_0.predict(X)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
h_0 = DecisionTreeRegressor(max_depth=2)
h_0.fit(X,y_0)

Our first ensemble is now simply the sum of $F_0$ and $h_0$: $$F_1 = F_0 + h_0$$

We take a look at its predictions on some new data:

In [ ]:
# A convenience function for plotting predictions of our ensembles
def plot_predictions(models):
    y_pred = sum(model.predict(X_new) for model in models)
    fig, ax = plt.subplots()
    ax.plot(X[:, 0], y, 'b.')
    ax.plot(X_new, y_pred, 'g-')
    plt.show()

In [ ]:
plot_predictions([F_0, h_0])

A big improvement!

Let's make a larger ensemble:

In [ ]:
# Residuals from previous ensemble
F_1_pred = F_0.predict(X) + h_0.predict(X) 
y1 = y - F_1_pred

In [ ]:
# Model fitted to the residuals:
h_1 = DecisionTreeRegressor(max_depth=2)
h_1.fit(X,y1)

In [ ]:
# Updated ensemble:
F_2_pred = F_1_pred + h_1.predict(X)

In [ ]:
# Residuals:
y2 = y - F_2_pred

In [ ]:
# Model fitted to the residuals:
h_2 = DecisionTreeRegressor(max_depth=2)
h_2.fit(X, y2)

In [ ]:
# Updated ensemble:
F_3_pred = F_2_pred + h_2.predict(X)

Here are the initial ensemble results:

In [ ]:
plot_predictions([F_0,])

In [ ]:
plot_predictions([F_0, h_0])

After adding a second model:

In [ ]:
plot_predictions([F_0, h_0, h_1])

After adding another model:

In [ ]:
plot_predictions([F_0, h_0, h_1, h_2])

**Magic!**

We can continue this for as long as we want:

In [ ]:
def boosting(k=3, X=X, y=y):
    """
    Trains k trees on the randomly generated points
    """

    # Construct models
    models = []
    
    ## Initial model:
    F_0 = DummyRegressor(strategy='mean')
    F_0.fit(X,y)
    y = y - F_0.predict(X)
    models.append(F_0)
    
    ## Creating the ensemble
    for i in range(k):
        h = DecisionTreeRegressor(max_depth=2)
        h.fit(X,y)
        models.append(h)
        y = y - h.predict(X)
    
    plot_predictions(models)

Here's an interactive plot showcasing the process:

In [ ]:
from ipywidgets import interactive, IntSlider, fixed

In [ ]:
interactive_plot = interactive(boosting, X=fixed(X), y=fixed(y),
                              k = IntSlider(min=0, max=50, step=1, value=0))
interactive_plot

#### Residual errors and the boosting algorithm:

The above recursive procedure can be described as follows:

* Create an initial model, $F_0$
* Fit a model to the residuals, $h_0(x) \sim y - F_0(x)$
* Create an ensemble, $F_1(x) = F_0(x) + h_0(x)$
* Fit a model to the residuals, $h_1(x) \sim y - F_1(x)$
* Create an ensemble, $F_2(x) = F_1(x) + h_1(x)$
* ... continue until reaching a stopping criterion

### Where's the gradient in gradient boosting?

It turns out that mathematically, the above procedure is actually doing gradient descent using the mean squared error loss function. 

The mean squared error loss function is

$$L(y, \hat{y}) = \frac 1m \sum_{i=1}^m (y_i - \hat{y}_i)^2.$$

Its partial derivatives with respect to the $\hat{y}_j$ are 

\begin{align}
\frac{\partial}{\partial \hat{y}_j} L(y,\hat{y}) &= \frac1m \frac{\partial}{\partial \hat{y}_j} (y_j - \hat{y}_j) \\ 
&= -\frac2m(y_j - \hat{y}_j)
\end{align}

The gradient is, therefore

$$\nabla_{\hat{y}} L(y,\hat{y}) = -\frac2m (y - \hat{y})$$

When we're doing $F_m(x) = F_{m-1}(x) + h_m(x)$ where $h_m(x)$ is a model fitted to the residuals $y-\hat{y}$ of the previous model, we're actually doing a gradient descent on the mean squared error loss function!

More precisely, when we're adding a new model to the ensemble, we're adding the negative of the gradient of the mean squared error loss function up to a constant (called the learning rate in both gradient descent and in gradient boosting).

This view of things provides the fundamental insight behind gradient boosting: we can swap out the mean squared loss function with any other (differentiable) loss function $L(y, \hat{y})$!

See [here](https://www.gormanalysis.com/blog/gradient-boosting-explained) for further details.

### Gradient boosting in scikit-learn

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
#??GradientBoostingRegressor

Let's try it on the housing data:

In [ ]:
gbrt = GradientBoostingRegressor(max_depth=3, n_estimators=20, 
                                 learning_rate=1.0, random_state=seed)

In [ ]:
gbrt.fit(X_train, y_train)

In [ ]:
mean_squared_error(y_test, gbrt.predict(X_test))

# Let's vote!

To illustrate model averaging through voting, let's ensemble a few models and have them vote:

In [ ]:
model1 = RandomForestRegressor(random_state=seed, n_jobs=-1)

model2 = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=None), 
                           n_estimators=50, random_state=seed)

model3 = GradientBoostingRegressor(max_depth=3, n_estimators=20, learning_rate=1.0, 
                                  random_state=seed)

In [ ]:
from sklearn.ensemble import VotingRegressor

In [ ]:
#?VotingRegressor

In [ ]:
ensemble = VotingRegressor([('rf', model1), 
                            ('ada', model2),
                            ('gb', model3)])

In [ ]:
ensemble.fit(X_train, y_train)

In [ ]:
mean_squared_error(y_test, ensemble.predict(X_test))

***Better than all the models used on their own!***

In [ ]:
for m in [model1, model2, model3]:
    m.fit(X_train, y_train)
    mse = mean_squared_error(y_test, m.predict(X_test))
    print(f'MSE: {mse}\n')

# Other advanced boosting techniques and libraries worth knowing about

There are multiple libraries for gradient boosting that can provide significantly more powerful models than scikit-learn. Two extremely widely used ones (you've undoubtedly come across them on Kaggle) are

- XGBoost: A widely used implementation of gradient boosting. Here are some sources for information: https://xgboost.readthedocs.io/en/latest, https://www.analyticsvidhya.com/blog/2018/09/an-end-to-end-guide-to-understand-the-math-behind-xgboost, https://campus.datacamp.com/courses/extreme-gradient-boosting-with-xgboost/classification-with-xgboost
- LightGBM: Another often-used gradient boosting library (in fact it's recently become more popular than XGBoost on Kaggle). Have a look at the documentation here: https://lightgbm.readthedocs.io/en/latest/index.html

If you want to use **stacking** where multiple model predictions are combined by training a *blender* (instead of simply voting), take a look at scikit-learn's `StackingClassifier` and `StackingRegressor`, or ML-Ensemble https://github.com/flennerhag/mlens and vecstack https://github.com/vecxoz/vecstack.

Here's an illustration of the idea from the textbook by A. Geron:

<img width=30% src="https://github.com/alu042/DAT158-2022/raw/main/notebooks/assets/stack.png">